# Deep Learning
In this notebook, we will be using neural networks in order to try and predict individual games. We will be using the data that was collected from the sportsreference API from the summer of 2020.

# Imports
Import the necessary modules we will be utilizing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

pd.set_option("display.max_rows", None, "display.max_columns", None)

# Getting Data Ready

In [2]:
# Load data.csv
dataframe = pd.read_csv('../assets/data/data.csv')
dataframe.head()

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rating,away_defensive_rebound_percentage,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,away_field_goals,away_free_throw_attempt_rate,away_free_throw_attempts,away_free_throw_percentage,away_free_throws,away_losses,away_minutes_played,away_offensive_rating,away_offensive_rebound_percentage,away_offensive_rebounds,away_personal_fouls,away_points,away_ranking,away_steal_percentage,away_steals,away_three_point_attempt_rate,away_three_point_field_goal_attempts,away_three_point_field_goal_percentage,away_three_point_field_goals,away_total_rebound_percentage,away_total_rebounds,away_true_shooting_percentage,away_turnover_percentage,away_turnovers,away_two_point_field_goal_attempts,away_two_point_field_goal_percentage,away_two_point_field_goals,away_win_percentage,away_wins,date,home_assist_percentage,home_assists,home_block_percentage,home_blocks,home_defensive_rating,home_defensive_rebound_percentage,home_defensive_rebounds,home_effective_field_goal_percentage,home_field_goal_attempts,home_field_goal_percentage,home_field_goals,home_free_throw_attempt_rate,home_free_throw_attempts,home_free_throw_percentage,home_free_throws,home_losses,home_minutes_played,home_offensive_rating,home_offensive_rebound_percentage,home_offensive_rebounds,home_personal_fouls,home_points,home_ranking,home_steal_percentage,home_steals,home_three_point_attempt_rate,home_three_point_field_goal_attempts,home_three_point_field_goal_percentage,home_three_point_field_goals,home_total_rebound_percentage,home_total_rebounds,home_true_shooting_percentage,home_turnover_percentage,home_turnovers,home_two_point_field_goal_attempts,home_two_point_field_goal_percentage,home_two_point_field_goals,home_win_percentage,home_wins,location,losing_abbr,losing_name,pace,winner,winning_abbr,winning_name
0,25.0,3,6.3,3,115.4,69.2,27,0.286,49,0.245,12,0.367,18,0.611,11,0,200,50.0,18.9,7,17,39,NaN,3.8,3,0.429,21,0.190,4,44.7,34,0.339,33.0,28,28,0.286,8,0.00,0,"November 5, 2019",55.6,20,7.1,2,50.0,81.1,30,0.520,75,0.480,36,0.227,17,0.706,12,0,200,115.4,30.8,12,21,90,NaN,20.5,16,0.360,27,0.222,6,55.3,42,0.542,7.8,7,48,0.625,30,1.000,1,"Moody Coliseum , Abilene, Texas",Arlington Baptist\r\n\t\t\t,Arlington Baptist\r\n\t\t\t,78.3,Home,ABILENE-CHRISTIAN,Abilene Christian
1,43.3,13,6.3,2,107.5,65.2,15,0.531,65,0.462,30,0.308,20,0.700,14,1,225,103.8,8.8,3,26,83,NaN,15.0,12,0.385,25,0.360,9,31.6,18,0.557,9.8,8,40,0.525,21,0.50,1,"November 10, 2019",55.6,15,12.5,5,103.8,91.2,31,0.596,52,0.519,27,0.615,32,0.750,24,1,225,107.5,34.8,8,17,86,NaN,5.0,4,0.385,20,0.400,8,68.4,39,0.640,25.0,22,32,0.594,19,0.667,2,"Daskalakis Athletic Center, Philadelphia, Penn...",ABILENE-CHRISTIAN,Abilene Christian,71.4,Home,DREXEL,Drexel
2,39.1,9,3.1,1,94.5,70.0,21,0.520,51,0.451,23,0.451,23,0.870,20,1,200,100.0,18.5,5,22,73,NaN,12.3,9,0.412,21,0.333,7,45.6,26,0.589,21.8,17,30,0.533,16,0.75,3,"November 16, 2019",45.0,9,0.0,0,100.0,81.5,22,0.450,50,0.400,20,0.660,33,0.727,24,2,200,94.5,30.0,9,20,69,NaN,13.7,10,0.360,18,0.278,5,54.4,31,0.525,18.9,15,32,0.469,15,0.333,1,"Moody Coliseum , Abilene, Texas",ABILENE-CHRISTIAN,Abilene Christian,72.8,Away,PEPPERDINE,Pepperdine
3,72.7,16,3.6,1,112.5,69.6,16,0.432,59,0.373,22,0.153,9,0.778,7,3,200,90.6,29.0,9,17,58,NaN,9.4,6,0.441,26,0.269,7,46.3,25,0.458,14.9,11,33,0.455,15,0.25,1,"November 18, 2019",51.9,14,12.1,4,90.6,71.0,22,0.600,50,0.540,27,0.300,15,0.800,12,3,200,112.5,30.4,7,13,72,NaN,6.3,4,0.440,22,0.273,6,53.7,29,0.630,17.5,12,28,0.750,21,0.400,2,"Thomas & Mack Center, Las Vegas, Nevada",ABILENE-CHRISTIAN,Abilene Christian,63.7,Home,NEVADA-LAS-VEGAS,UNLV
4,43.8,7,2.4,1,121.6,55.6,20,0.433,45,0.356,16,0.467,21,0.905,19,0,200,78.4,17.2,5,23,58,NaN,9.5,7,0.356,16,0.438,7,38.5,25,0.528,29.8,23,29,0.310,9,0.00,0,"November 21, 2019",66.7,22,10.3,3,78.4,82.8,24,0.545,67,0.493,33,0.343,23,0.739,17,3,200,121.6,44.4,16,17,90

In [3]:
# These fields aren't necessary to train the model
FIELDS_TO_DROP = ['date',
                  'location',
                  'away_defensive_rating',
                  'home_defensive_rating',
                  'away_defensive_rebound_percentage',
                  'home_defensive_rebound_percentage',
                  'losing_abbr',
                  'winner',
                  'winning_abbr',
                  'home_ranking',
                  'away_ranking',
                  'winning_name',
                  'losing_name',
                  'away_points',
                  'home_points']
data = dataframe.copy()
data['target'] = np.where(dataframe['winner'] == "Home", 0, 1)
data.drop(columns=FIELDS_TO_DROP, inplace=True)
data.dropna(inplace=True)
data.head()

,away_assist_percentage,away_assists,away_block_percentage,away_blocks,away_defensive_rebounds,away_effective_field_goal_percentage,away_field_goal_attempts,away_field_goal_percentage,away_field_goals,away_free_throw_attempt_rate,away_free_throw_attempts,away_free_throw_percentage,away_free_throws,away_losses,away_minutes_played,away_offensive_rating,away_offensive_rebound_percentage,away_offensive_rebounds,away_personal_fouls,away_steal_percentage,away_steals,away_three_point_attempt_rate,away_three_point_field_goal_attempts,away_three_point_field_goal_percentage,away_three_point_field_goals,away_total_rebound_percentage,away_total_rebounds,away_true_shooting_percentage,away_turnover_percentage,away_turnovers,away_two_point_field_goal_attempts,away_two_point_field_goal_percentage,away_two_point_field_goals,away_win_percentage,away_wins,home_assist_percentage,home_assists,home_block_percentage,home_blocks,home_defensive_rebounds,home_effective_field_goal_percentage,home_field_goal_attempts,home_field_goal_percentage,home_field_goals,home_free_throw_attempt_rate,home_free_throw_attempts,home_free_throw_percentage,home_free_throws,home_losses,home_minutes_played,home_offensive_rating,home_offensive_rebound_percentage,home_offensive_rebounds,home_personal_fouls,home_steal_percentage,home_steals,home_three_point_attempt_rate,home_three_point_field_goal_attempts,home_three_point_field_goal_percentage,home_three_point_field_goals,home_total_rebound_percentage,home_total_rebounds,home_true_shooting_percentage,home_turnover_percentage,home_turnovers,home_two_point_field_goal_attempts,home_two_point_field_goal_percentage,home_two_point_field_goals,home_win_percentage,home_wins,pace,target
0,25.0,3,6.3,3,27,0.286,49,0.245,12,0.367,18,0.611,11,0,200,50.0,18.9,7,17,3.8,3,0.429,21,0.190,4,44.7,34,0.339,33.0,28,28,0.286,8,0.00,0,55.6,20,7.1,2,30,0.520,75,0.480,36,0.227,17,0.706,12,0,200,115.4,30.8,12,21,20.5,16,0.360,27,0.222,6,55.3,42,0.542,7.8,7,48,0.625,30,1.000,1,78.3,0
1,43.3,13,6.3,2,15,0.531,65,0.462,30,0.308,20,0.700,14,1,225,103.8,8.8,3,26,15.0,12,0.385,25,0.360,9,31.6,18,0.557,9.8,8,40,0.525,21,0.50,1,55.6,15,12.5,5,31,0.596,52,0.519,27,0.615,32,0.750,24,1,225,107.5,34.8,8,17,5.0,4,0.385,20,0.400,8,68.4,39,0.640,25.0,22,32,0.594,19,0.667,2,71.4,0
2,39.1,9,3.1,1,21,0.520,51,0.451,23,0.451,23,0.870,20,1,200,100.0,18.5,5,22,12.3,9,0.412,21,0.333,7,45.6,26,0.589,21.8,17,30,0.533,16,0.75,3,45.0,9,0.0,0,22,0.450,50,0.400,20,0.660,33,0.727,24,2,200,94.5,30.0,9,20,13.7,10,0.360,18,0.278,5,54.4,31,0.525,18.9,15,32,0.469,15,0.333,1,72.8,1
3,72.7,16,3.6,1,16,0.432,59,0.373,22,0.153,9,0.778,7,3,200,90.6,29.0,9,17,9.4,6,0.441,26,0.269,7,46.3,25,0.458,14.9,11,33,0.455,15,0.25,1,51.9,14,12.1,4,22,0.600,50,0.540,27,0.300,15,0.800,12,3,200,112.5,30.4,7,13,6.3,4,0.440,22,0.273,6,53.7,29,0.630,17.5,12,28,0.750,21,0.400,2,63.7,0
4,43.8,7,2.4,1,20,0.433,45,0.356,16,0.467,21,0.905,19,0,200,78.4,17.2,5,23,9.5,7,0.356,16,0.438,7,38.5,25,0.528,29.8,23,29,0.310,9,0.00,0,66.7,22,10.3,3,24,0.545,67,0.493,33,0.343,23,0.739,17,3,200,121.6,44.4,16,17,8.1,6,0.373,25,0.280,7,61.5,40,0.577,14.4,13,42,0.619,26,0.400,2,74.0,0


Now, we have to split the data into training, validation, and test sets

In [4]:
train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7075 train examples
1769 validation examples
2212 test examples


Next, we need wrap the dataframes with `tf.data`, in order to shuffle and batch the data.

In [5]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

For each of the Numeric feature, we will use a Normalization() layer to make sure the mean of each feature is 0 and its standard deviation is 1.

In [6]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for our feature.
    normalizer = preprocessing.Normalization()

    # Prepare a Dataset that only yields our feature.
    feature_ds = dataset.map(lambda x, y: x[name])

    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)

    return normalizer

Create new input pipeline and make numeric columns for all of the features

In [7]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [8]:
all_inputs = []
encoded_features = []

# Just to get the training features
[(train_features, label_batch)] = train_ds.take(1)

# Numeric features.
for header in tqdm(list(train_features.keys()), unit='feautures'):
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_inputs.append(numeric_col)
    encoded_features.append(encoded_numeric_col)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [00:21<00:00,  3.35feautures/s]


# Compile the Neural Network

In [9]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(16, activation="relu")(all_features)
x = tf.keras.layers.Dense(16, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

# Train the Network

In [10]:
model.fit(train_ds, epochs=5, validation_data=val_ds)

Epoch 1/5
222/222 [==============================] - 5s 11ms/step - loss: 0.5416 - accuracy: 0.7104 - val_loss: 0.1787 - val_accuracy: 0.9259
Epoch 2/5
222/222 [==============================] - 1s 3ms/step - loss: 0.1791 - accuracy: 0.9324 - val_loss: 0.0940 - val_accuracy: 0.9650
Epoch 3/5
222/222 [==============================] - 1s 3ms/step - loss: 0.1085 - accuracy: 0.9612 - val_loss: 0.0689 - val_accuracy: 0.9734
Epoch 4/5
222/222 [==============================] - 1s 3ms/step - loss: 0.0786 - accuracy: 0.9730 - val_loss: 0.0527 - val_accuracy: 0.9813
Epoch 5/5
222/222 [==============================] - 1s 3ms/step - loss: 0.0604 - accuracy: 0.9810 - val_loss: 0.0430 - val_accuracy: 0.9830


In [11]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

70/70 [==============================] - 0s 2ms/step - loss: 0.0412 - accuracy: 0.9860
Accuracy 0.985985517501831


# Analysis
The results here, while they may look spectacular, are very misleading. The way that the current data is setup, we are only looking at the game stats and determining the winner based on that. If we have the number of 2PM, 3PM, and freethrows for both teams in the current game, it's trivial to find out who won the game, we can easily just calculate the number of points each team had. To make this better, we should restructure our data. In particular, for each sample (row), it should contain the **averages** for both teams **going into** the match up and then have a label of which team won after they played. This way, we don't know the game stats, but might be able to predict what they could be given the teams' averages.

# Take 2
Now we'll retrain a new neural network on the moving averages data that we generated.

In [12]:
data_ma = pd.read_csv('../assets/data/cleaned_data/cleaned_data.csv')
data_ma.head()

,date,location,home,away,winner,home_assist_percentage_SMA,home_assist_percentage_CMA,home_assist_percentage_EMA,home_assists_SMA,home_assists_CMA,home_assists_EMA,home_block_percentage_SMA,home_block_percentage_CMA,home_block_percentage_EMA,home_blocks_SMA,home_blocks_CMA,home_blocks_EMA,home_defensive_rating_SMA,home_defensive_rating_CMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_SMA,home_defensive_rebound_percentage_CMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_SMA,home_defensive_rebounds_CMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_SMA,home_effective_field_goal_percentage_CMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_SMA,home_field_goal_attempts_CMA,home_field_goal_attempts_EMA,home_field_goal_percentage_SMA,home_field_goal_percentage_CMA,home_field_goal_percentage_EMA,home_field_goals_SMA,home_field_goals_CMA,home_field_goals_EMA,home_free_throw_attempt_rate_SMA,home_free_throw_attempt_rate_CMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_SMA,home_free_throw_attempts_CMA,home_free_throw_attempts_EMA,home_free_throw_percentage_SMA,home_free_throw_percentage_CMA,home_free_throw_percentage_EMA,home_free_throws_SMA,home_free_throws_CMA,home_free_throws_EMA,home_losses_SMA,home_losses_CMA,home_losses_EMA,home_minutes_played_SMA,home_minutes_played_CMA,home_minutes_played_EMA,home_offensive_rating_SMA,home_offensive_rating_CMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_SMA,home_offensive_rebound_percentage_CMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_SMA,home_offensive_rebounds_CMA,home_offensive_rebounds_EMA,home_personal_fouls_SMA,home_personal_fouls_CMA,home_personal_fouls_EMA,home_points_SMA,home_points_CMA,home_points_EMA,home_steal_percentage_SMA,home_steal_percentage_CMA,home_steal_percentage_EMA,home_steals_SMA,home_steals_CMA,home_steals_EMA,home_three_point_attempt_rate_SMA,home_three_point_attempt_rate_CMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_SMA,home_three_point_field_goal_attempts_CMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_SMA,home_three_point_field_goal_percentage_CMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_SMA,home_three_point_field_goals_CMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_SMA,home_total_rebound_percentage_CMA,home_total_rebound_percentage_EMA,home_total_rebounds_SMA,home_total_rebounds_CMA,home_total_rebounds_EMA,home_true_shooting_percentage_SMA,home_true_shooting_percentage_CMA,home_true_shooting_percentage_EMA,home_turnover_percentage_SMA,home_turnover_percentage_CMA,home_turnover_percentage_EMA,home_turnovers_SMA,home_turnovers_CMA,home_turnovers_EMA,home_two_point_field_goal_attempts_SMA,home_two_point_field_goal_attempts_CMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_SMA,home_two_point_field_goal_percentage_CMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_SMA,home_two_point_field_goals_CMA,home_two_point_field_goals_EMA,home_win_percentage_SMA,home_win_percentage_CMA,home_win_percentage_EMA,home_wins_SMA,home_wins_CMA,home_wins_EMA,away_assist_percentage_SMA,away_assist_percentage_CMA,away_assist_percentage_EMA,away_assists_SMA,away_assists_CMA,away_assists_EMA,away_block_percentage_SMA,away_block_percentage_CMA,away_block_percentage_EMA,away_blocks_SMA,away_blocks_CMA,away_blocks_EMA,away_defensive_rating_SMA,away_defensive_rating_CMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_SMA,away_defensive_rebound_percentage_CMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_SMA,away_defensive_rebounds_CMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_SMA,away_effective_field_goal_percentage_CMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_SMA,away_field_goal_attempts_CMA,away_field_goal_attempts_EMA,away_field_goal_perc

In [13]:
# These fields aren't necessary to train the model
DROPPING = ['date',
            'location',
            'home',
            'away']

ma = data_ma.copy()
ma['target'] = np.where(ma['winner'] == "Home", 0, 1)
ma.drop(columns=DROPPING, inplace=True)
ma.dropna(inplace=True)
ma.drop_duplicates(inplace=True)
ma.pop('winner')

target = ma.pop('target')
ma = ma.filter(regex="EMA", axis=1)
ma['target'] = target
ma.head()

,home_assist_percentage_EMA,home_assists_EMA,home_block_percentage_EMA,home_blocks_EMA,home_defensive_rating_EMA,home_defensive_rebound_percentage_EMA,home_defensive_rebounds_EMA,home_effective_field_goal_percentage_EMA,home_field_goal_attempts_EMA,home_field_goal_percentage_EMA,home_field_goals_EMA,home_free_throw_attempt_rate_EMA,home_free_throw_attempts_EMA,home_free_throw_percentage_EMA,home_free_throws_EMA,home_losses_EMA,home_minutes_played_EMA,home_offensive_rating_EMA,home_offensive_rebound_percentage_EMA,home_offensive_rebounds_EMA,home_personal_fouls_EMA,home_points_EMA,home_steal_percentage_EMA,home_steals_EMA,home_three_point_attempt_rate_EMA,home_three_point_field_goal_attempts_EMA,home_three_point_field_goal_percentage_EMA,home_three_point_field_goals_EMA,home_total_rebound_percentage_EMA,home_total_rebounds_EMA,home_true_shooting_percentage_EMA,home_turnover_percentage_EMA,home_turnovers_EMA,home_two_point_field_goal_attempts_EMA,home_two_point_field_goal_percentage_EMA,home_two_point_field_goals_EMA,home_win_percentage_EMA,home_wins_EMA,away_assist_percentage_EMA,away_assists_EMA,away_block_percentage_EMA,away_blocks_EMA,away_defensive_rating_EMA,away_defensive_rebound_percentage_EMA,away_defensive_rebounds_EMA,away_effective_field_goal_percentage_EMA,away_field_goal_attempts_EMA,away_field_goal_percentage_EMA,away_field_goals_EMA,away_free_throw_attempt_rate_EMA,away_free_throw_attempts_EMA,away_free_throw_percentage_EMA,away_free_throws_EMA,away_losses_EMA,away_minutes_played_EMA,away_offensive_rating_EMA,away_offensive_rebound_percentage_EMA,away_offensive_rebounds_EMA,away_personal_fouls_EMA,away_points_EMA,away_steal_percentage_EMA,away_steals_EMA,away_three_point_attempt_rate_EMA,away_three_point_field_goal_attempts_EMA,away_three_point_field_goal_percentage_EMA,away_three_point_field_goals_EMA,away_total_rebound_percentage_EMA,away_total_rebounds_EMA,away_true_shooting_percentage_EMA,away_turnover_percentage_EMA,away_turnovers_EMA,away_two_point_field_goal_attempts_EMA,away_two_point_field_goal_percentage_EMA,away_two_point_field_goals_EMA,away_win_percentage_EMA,away_wins_EMA,target
0,60.748148,13.012346,4.704938,1.580247,90.790123,82.006173,26.987654,0.477049,53.641975,0.398296,21.555556,0.370296,19.555556,0.800123,15.148148,1.000000,199.802469,102.702469,23.656790,7.518519,13.407407,66.703704,7.507407,4.740741,0.493753,26.271605,0.318815,8.444444,53.501235,34.506173,0.527469,12.735802,9.185185,27.370370,0.463333,13.111111,0.581531,2.395062,49.120988,11.617284,13.429630,5.074074,86.544444,74.550617,25.592593,0.485864,58.802469,0.406642,23.370370,0.397914,21.740741,0.699975,15.370370,1.802469,204.938272,99.222222,28.197531,10.679012,13.481481,70.851852,8.940741,6.444444,0.451827,26.234568,0.343704,8.740741,52.322222,36.271605,0.522778,17.501235,14.148148,32.567901,0.460296,14.629630,0.360444,1.592593,1
1,47.316461,11.905350,9.581481,2.600823,86.714403,87.358436,26.843621,0.524333,54.086420,0.472276,25.654321,0.355288,19.345679,0.642840,12.074074,1.572016,200.000000,108.503292,32.730864,9.333333,15.477366,69.045267,10.812757,6.884774,0.286979,15.687243,0.364667,5.662551,60.924280,36.176955,0.543510,11.871193,8.460905,38.399177,0.522975,19.991770,0.660226,2.691358,38.283951,9.259259,7.653086,2.617284,100.492593,69.771605,24.814815,0.425185,66.000000,0.366716,24.098765,0.323642,21.086420,0.697506,14.827160,2.691358,200.000000,87.290123,21.643210,8.950617,22.987654,70.666667,9.046914,7.407407,0.468247,31.111111,0.255556,7.641975,43.813580,33.765432,0.465407,16.412346,14.716049,34.888889,0.472333,16.456790,0.171556,0.703704,0
2,58.086420,13.345679,8.462963,3.469136,96.674074,76.877778,24.938272,0.502827,53.950617,0.438494,22.740741,0.438173,20.567901,0.717667,14.790123,1.888889,203.703704,88.416049,24.940741,8.395062,17.901235,67.098765,8.839506,6.617284,0.388099,21.333333,0.335617,6.827160,51.477778,33.333333,0.536617,24.403704,20.345679,32.617284,0.502358,15.913580,0.392642,1.506173,55.643621,17.403292,8.8

In [14]:
train_ma, test_ma = train_test_split(ma, test_size=0.2)
train_ma, val_ma = train_test_split(train_ma, test_size=0.2)
print(len(train_ma), 'train examples')
print(len(val_ma), 'validation examples')
print(len(test_ma), 'test examples')

2248 train examples
562 validation examples
703 test examples


In [15]:
batch_size = 32
train_ma_ds = df_to_dataset(train_ma, batch_size=batch_size)
val_ma_ds = df_to_dataset(val_ma, shuffle=False, batch_size=batch_size)
test_ma_ds = df_to_dataset(test_ma, shuffle=False, batch_size=batch_size)

In [16]:
all_ma_inputs = []
encoded_ma_features = []

# Just to get the training features
[(train_ma_features, label_ma_batch)] = train_ma_ds.take(1)

# Numeric features.
for header in tqdm(list(train_ma_features.keys()), unit="features"):
    numeric_col = tf.keras.Input(shape=(1,), name=header)
    normalization_layer = get_normalization_layer(header, train_ma_ds)
    encoded_numeric_col = normalization_layer(numeric_col)
    all_ma_inputs.append(numeric_col)
    encoded_ma_features.append(encoded_numeric_col)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:11<00:00,  6.77features/s]


In [17]:
all_ma_features = tf.keras.layers.concatenate(encoded_ma_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_ma_features)
x = tf.keras.layers.Dense(32, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
output_ma = tf.keras.layers.Dense(1)(x)
model_ma = tf.keras.Model(all_ma_inputs, output_ma)
model_ma.compile(optimizer='adam',
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 metrics=["accuracy"])

In [18]:
model_ma.fit(train_ma_ds, epochs=20, validation_data=val_ma_ds)

Epoch 1/20
71/71 [==============================] - 3s 16ms/step - loss: 0.7351 - accuracy: 0.5803 - val_loss: 0.6336 - val_accuracy: 0.6317
Epoch 2/20
71/71 [==============================] - 0s 3ms/step - loss: 0.6544 - accuracy: 0.6154 - val_loss: 0.6212 - val_accuracy: 0.6423
Epoch 3/20
71/71 [==============================] - 1s 7ms/step - loss: 0.6154 - accuracy: 0.6707 - val_loss: 0.6119 - val_accuracy: 0.6423
Epoch 4/20
71/71 [==============================] - 0s 4ms/step - loss: 0.6020 - accuracy: 0.6610 - val_loss: 0.6044 - val_accuracy: 0.6441
Epoch 5/20
71/71 [==============================] - 0s 3ms/step - loss: 0.6067 - accuracy: 0.6627 - val_loss: 0.6088 - val_accuracy: 0.6512
Epoch 6/20
71/71 [==============================] - 0s 3ms/step - loss: 0.5921 - accuracy: 0.6599 - val_loss: 0.6043 - val_accuracy: 0.6601
Epoch 7/20
71/71 [==============================] - 0s 3ms/step - loss: 0.5791 - accuracy: 0.6690 - val_loss: 0.6039 - val_accuracy: 0.6637
Epoch 8/20
71/71 [=

In [19]:
loss_ma, accuracy_ma = model_ma.evaluate(test_ma_ds)
print("Accuracy", accuracy_ma)

22/22 [==============================] - 0s 2ms/step - loss: 0.6199 - accuracy: 0.6799
Accuracy 0.6799430847167969


# Analysis on the New Model
Overall, a 68% accuracy, especially when it comes to sports predictions is quite impressive. This mean that the model will correctly determine the winner of a game two out of three times. It's unreasonable to expect a very high accuracy due to the randomness in sports, however this is a really great start.

# Using the Model for Singular Inputs

In [20]:
sample = test_ma.drop(columns=['target']).to_dict("list")

input_dict = {name: tf.convert_to_tensor([value[0]]) for name, value in sample.items()}
predictions = model_ma.predict(input_dict)
prob = tf.nn.sigmoid(predictions[0])

print("For this particular sample, there is %.1f percent probability of the away team winning." % (100 * prob))

For this particular sample, there is 23.4 percent probability of the away team winning.
